In [227]:
import pandas as pd 
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
import torch.nn as nn
import random

#When having mismatch between lib versions in Jupyter, RELOAD THE KERNEL!!
#https://stackoverflow.com/questions/61366994/jupyter-notebook-does-not-reflect-library-version-change-in-virtualenvironment

#https://www.kaggle.com/datasets/kritanjalijain/amazon-reviews
#1-Negative
#2-Positive

In [256]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [471]:
df=pd.read_csv("train.csv")

In [472]:
df=df.dropna()
df=df.replace({'\n': ' '})

In [473]:
df.head(n=2)

,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^
0,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
1,2,Amazing!,This soundtrack is my favorite music of all ti...


In [474]:
ratings=df.iloc[:, 0]
ratings=[[0, 1] if r==2 else [1,0] for r in ratings] #[0,1]-Positive, [1,0]-Negative
ratings=np.float64(np.array(ratings))
print (ratings[:10])

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [475]:
reviews=df.iloc[:, 1:]
reviews=reviews.to_numpy()
reviews=reviews[:, 0]+' '+reviews[:, 1]

In [476]:
print(str(reviews[1][:200])+'...')

Amazing! This soundtrack is my favorite music of all time, hands down. The intense sadness of "Prisoners of Fate" (which means all the more if you've played the game) and the hope in "A Distant Promis...


In [478]:
#nltk.download('punkt')
reviews=reviews[:10000] #First 10000 elements
ratings=ratings[:10000]
tokens=[]
for i in range(len(reviews)):
    token=word_tokenize(reviews[i])
    tokens.append(token)
print(tokens[0][:20],'...')

['The', 'best', 'soundtrack', 'ever', 'to', 'anything', '.', 'I', "'m", 'reading', 'a', 'lot', 'of', 'reviews', 'saying', 'that', 'this', 'is', 'the', 'best'] ...


In [479]:
print (len(reviews), len(ratings))

10000 10000


In [480]:
review_vecs=[]
for token in tokens:
    vecs=[]
    for tk in token:
        if tk not in model:
            continue
        vec=model[tk]
        vecs.append(vec)
        
    vecs=vecs[:100]
    for i in range(len(vecs), 100):
        vecs.append(np.zeros(shape=[300]))
    review_vecs.append(vecs)

In [481]:
review_vecs=np.array(review_vecs)
#review_vecs=np.float32(review_vecs)
print (review_vecs.shape, review_vecs.dtype, len(ratings))

(10000, 100, 300) float64 10000


In [482]:
max=len(review_vecs)
print(review_vecs[random.randint(0, max)])

[[-1.05957031e-01  2.13867188e-01  1.18652344e-01 ...  1.06933594e-01
   2.36816406e-02 -3.54003906e-02]
 [ 1.37695312e-01 -1.79687500e-01 -3.27148438e-02 ... -2.85156250e-01
  -1.42578125e-01 -9.96093750e-02]
 [ 2.38281250e-01  1.51367188e-01 -6.44531250e-02 ... -4.45312500e-01
   6.39648438e-02 -1.90734863e-04]
 ...
 [ 3.84765625e-01  7.22656250e-02  1.83593750e-01 ... -6.10351562e-02
  -2.37304688e-01  1.52343750e-01]
 [-1.69921875e-01  2.01171875e-01  2.05078125e-01 ...  1.79687500e-01
   2.11914062e-01 -1.86523438e-01]
 [ 7.04956055e-03 -7.32421875e-02  1.71875000e-01 ...  1.12304688e-02
   1.64062500e-01  1.06933594e-01]]


In [494]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [500]:
class AmazonDataset(Dataset):
    def __init__(self, vecs, rating, device, transform=None):
        self.review_vecs=torch.from_numpy(vecs).to(device)
        self.ratings=torch.from_numpy(rating).to(device)
    def __len__(self):
        return len(ratings)
    def __getitem__(self, idx):
        return self.review_vecs[idx].reshape(1, -1), self.ratings[idx]

In [501]:
train_dataset=AmazonDataset(review_vecs, ratings, device)
train_datalod=DataLoader(train_dataset, shuffle=True, batch_size=1000)

In [502]:
len(train_dataset)

10000

In [503]:
class ReviewModel(nn.Module):
    def __init__(self, n_hidden):
        super().__init__()
        self.l1=nn.Linear(30000, n_hidden).double()
        self.lrl=nn.LeakyReLU().double()
        self.l2=nn.Linear(n_hidden, n_hidden).double()
        self.rl=nn.ReLU().double()
        self.l3=nn.Linear(n_hidden, 2).double()
    def forward(self, x):
        x=self.lrl(self.l1(x))
        x=self.lrl(self.l2(x))
        x=self.l3(x)
        return x

In [512]:
rmodel=ReviewModel(1000).to(device)
optimizer=torch.optim.SGD(params=rmodel.parameters(), lr=0.001)
loss_fn=nn.CrossEntropyLoss()
n_epochs=20

In [513]:
for epoch in range(n_epochs):
    i=0
    for X, y in train_datalod:
        #print(X)
        #print(torch.isnan(X).any(), torch.isnan(y).any(), torch.isnan(y_pred).any())
        y_pred=rmodel(X)
        y_pred=y_pred.reshape(1, -1)
        y=y.reshape(1, -1)
        #print(X.shape, y_pred.shape, y.shape)
        #print(X)
        #print("X:", X)
        #print("y_pred:", y_pred)
        #print("y:", y)
        #break
        loss=loss_fn(y_pred, y).double()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i%2==0:
            #print(y_pred)
            print (f"Epoch {epoch+1}, Index: {i}, Loss: {loss}")
        i+=1

Epoch 1, Index: 0, Loss: 7600.856730059095
Epoch 1, Index: 2, Loss: 7597.225043460961
Epoch 1, Index: 4, Loss: 7592.27988604401
Epoch 1, Index: 6, Loss: 7589.778647027404
Epoch 1, Index: 8, Loss: 7580.354433091226
Epoch 2, Index: 0, Loss: 7581.828136552391
Epoch 2, Index: 2, Loss: 7696.810148941853
Epoch 2, Index: 4, Loss: 7596.042124149048
Epoch 2, Index: 6, Loss: 7586.85599649416
Epoch 2, Index: 8, Loss: 7574.198728080288
Epoch 3, Index: 0, Loss: 7545.489405423268
Epoch 3, Index: 2, Loss: 7800.267648590223
Epoch 3, Index: 4, Loss: 7598.025295410759
Epoch 3, Index: 6, Loss: 7592.370931679016
Epoch 3, Index: 8, Loss: 7583.972786486422
Epoch 4, Index: 0, Loss: 7564.004813356671
Epoch 4, Index: 2, Loss: 7535.126472695964
Epoch 4, Index: 4, Loss: 7713.495865553624
Epoch 4, Index: 6, Loss: 7608.581067831043
Epoch 4, Index: 8, Loss: 7602.955243982084


KeyboardInterrupt: 

In [378]:
!pip install scipy==1.10.1
!pip install --upgrade nltk
!pip install --upgrade gensim